# FEMA
Test implementation of FEMA (Fast and efficient mixed-effects algorithm) in Python

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
import scipy.stats as st
import scipy.optimize as opt


In [ ]:
np.random.seed(1234)

## Create dataset
Forward calculations of data

In [ ]:
J = 500  # sample size / number of measures
N = 100  # number of observations


In [ ]:

# distributions from which random variates are drawn.
dist_beta = st.uniform(loc=-0.02, scale=0.04)  # fixed effects
dist_F = st.uniform(loc=0.2, scale=0.6)  # family
dist_S = st.uniform(loc=0.2, scale=0.6)  # subject
dist_E = st.norm(loc=0, scale=np.sqrt(1 - dist_F.var() - dist_S.var()))  # independent errors, assuming var_jF + var_jS + var_jE = 1;

dist_X = st.norm(loc=0, scale=1)
dist_mu = st.norm(loc=0, scale=1)

In [ ]:
# variances
var_F, var_S, var_E = dist_F.rvs(J), dist_S.rvs(J), dist_E.rvs(J)

In [ ]:
# compute V:
P = 0.1
# familiarity and subject random intercept matrices
Z_F = np.random.binomial(1, P, size=J * N).reshape((J, N))
Z_S = np.random.binomial(1, P, size=J * N).reshape((J, N))

V = var_F @ Z_F @ Z_F.T + var_S @ Z_F @ Z_F.T + var_E @ np.eye(J)
V = V.reshape((-1, 1))
V.shape

In [ ]:
# Construct data
X = dist_X.rvs(N * J).reshape((J, N))
beta = dist_beta.rvs(N).reshape((-1, 1))
mu = dist_mu.rvs(J).reshape((-1, 1)) * V
Y = X @ beta + mu

In [ ]:
X.shape, beta.shape, mu.shape, Y.shape, V.shape

In [ ]:
# plot Y
plt.plot(Y)

# FEMA
implement algorithm as described in Fan et al. 2021

In [ ]:


# Y = np.random.randn(J).reshape((J, 1))
# Y_hat = np.random.randn(J).reshape((J, 1))

# compute residual vector
B_hat = np.linalg.inv(X.T @ X) @ X.T @ Y
Y_hat_res = Y - X @ B_hat

In [ ]:
V_inv = np.linalg.inv(V)
var_beta_hat = np.linalg.inv(X.T @ V_inv @ X)
beta_hat = var_beta_hat @ X.T @ V_inv @ Y

In [ ]:
# Expectation of product of residuals Y_res
E_res = var_F @ (Z_F @ Z_F.T) + var_S @ (Z_F @ Z_F.T) + var_E

In [ ]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

# Simulate data for illustration
group_size = 5
n_groups = 100
x1 = np.random.normal(size=group_size*n_groups)
x2 = np.random.normal(size=group_size*n_groups)
u = np.kron(np.random.normal(size=n_groups), np.ones(group_size))
g = np.kron(np.arange(n_groups), np.ones(group_size))
e = np.random.normal(size=group_size*n_groups)
y = x1 - x2 + u + e

In [ ]:
plt.plot(u)
plt.plot(g)

In [ ]:

# Fit a multilevel model
df = pd.DataFrame({"y":y, "x1":x1, "x2":x2, "g":g})
model = sm.MixedLM.from_formula("y ~ x1 + x2", groups="g", data=df)
result = model.fit()

# The BLUPs
re = result.random_effects

# Multiply each BLUP by the random effects design matrix for one group
rex = [np.dot(model.exog_re_li[j], re[k]) for (j, k) in enumerate(model.group_labels)]

# Add the fixed and random terms to get the overall prediction
rex = np.concatenate(rex)
yp = result.fittedvalues + rex

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(rex)
plt.plot(x1 + x2)

In [ ]:
plt.figure(figsize=(16, 9))
plt.plot(y, label='y')
plt.plot(result.fittedvalues, label='y-fit')
plt.plot(yp, label='y-pred')